# Preprocess

This notebook handles pre-processing all the data to make sure it is consistent and there are no gaps in it.

Note: This notebook assumes that you have loaded data within the `data/portfolios.json` and `data/pricing` by running the `init.py`.


In [34]:
# Lib imports.
import pandas as pd
import os
import numpy as np

# Typing.
from typing import Optional, Tuple

In [35]:
# Constants
DATA_PATH_IN = "data/raw"
DATA_PATH_OUT = "data/pricing"
FUNDS_PATH = "data/portfolios.json"

# Columns.
COLUMN_DATE = "asOfDate"
COLUMN_PRICE = "price"

## Definition

This section defines all the functions we need for later.


In [36]:
def load_data(path: str) -> Optional[pd.DataFrame]:
    """Loads the data as per the specification of the pricing CSVs.

    Args:
        path (str): to the current file to load.

    Returns:
        Optional[pd.DataFrame]: with the pricing data sorted by date, ascending. None if there is no data.
    """
    # Make sure it is a CSV and not something else.
    if ".csv" not in path:
        return None

    # Loads the data.
    df = pd.read_csv(
        path,
        header=0,
        index_col=COLUMN_DATE,
        parse_dates=[COLUMN_DATE],
        dtype={
            "price": np.float64,
            "currencyCode": np.string_,
            "__typename": np.string_,
        },
    )

    # Sort by the index that we defined as date.
    return df.sort_index(ascending=True, inplace=False)

In [37]:
def fill_gaps(df: pd.DataFrame) -> pd.DataFrame:
    """Fills the gaps in the pricing data by backfilling from the previous existing dates.

    Args:
        df (pd.DataFrame): with potential gaps in the date index.

    Returns:
        pd.DataFrame: with no gaps in the index.
    """
    # Get all the possible dates.
    date_min, date_max = (df.index.min(), df.index.max())

    # We reindex using the min and max dates. This will propagate the values.
    return df.reindex(pd.date_range(start=date_min, end=date_max), method="ffill")

## Testing

This section tests the functions defined above.

In [38]:
def evaluate_index_size(df: pd.DataFrame, verbose:bool = True) -> Tuple[int, int]:
    # We can expect the number of rows to be less than a range of dates.
    full_range_len = len(pd.date_range(start=df.index.min(), end=df.index.max()))
    index_len = df.index.shape[0]

    if verbose:
        print(f"Full Size vs Index Size = {full_range_len} vs {index_len}")
    return full_range_len, index_len

In [39]:
EXAMPLE_DATA = "data/raw/8617.csv"

df = load_data(EXAMPLE_DATA)

# We can verify that there a gap since the gaps occur in weekends where the market is closed.
print(df)

# Make sure we assert it.
full_size, index_size = evaluate_index_size(df)
assert full_size > index_size

               price currencyCode __typename
asOfDate                                    
2016-11-08  100.0000          GBP  FundPrice
2016-11-09   99.7742          GBP  FundPrice
2016-11-10   99.7799          GBP  FundPrice
2016-11-11   98.5293          GBP  FundPrice
2016-11-14   99.3307          GBP  FundPrice
...              ...          ...        ...
2024-10-07  218.3068          GBP  FundPrice
2024-10-08  218.1539          GBP  FundPrice
2024-10-09  219.3834          GBP  FundPrice
2024-10-10  219.6460          GBP  FundPrice
2024-10-11  220.5142          GBP  FundPrice

[2002 rows x 3 columns]
Full Size vs Index Size = 2895 vs 2002


In [40]:
df = fill_gaps(df)

# We can verify that now there shouldn't be any gaps.
print(df)

# Make sure we assert it.
full_size, index_size = evaluate_index_size(df)
assert full_size == index_size

               price currencyCode __typename
2016-11-08  100.0000          GBP  FundPrice
2016-11-09   99.7742          GBP  FundPrice
2016-11-10   99.7799          GBP  FundPrice
2016-11-11   98.5293          GBP  FundPrice
2016-11-12   98.5293          GBP  FundPrice
...              ...          ...        ...
2024-10-07  218.3068          GBP  FundPrice
2024-10-08  218.1539          GBP  FundPrice
2024-10-09  219.3834          GBP  FundPrice
2024-10-10  219.6460          GBP  FundPrice
2024-10-11  220.5142          GBP  FundPrice

[2895 rows x 3 columns]
Full Size vs Index Size = 2895 vs 2895


## Propagation

Now we perform the propagation against ALL the data in our pricing.

In [41]:
files = [
    os.path.join(DATA_PATH_IN, file)
    for file in os.listdir(DATA_PATH_IN)
    if os.path.isfile(os.path.join(DATA_PATH_IN, file)) and ".csv" in file
]

print(files)

['data/raw/E059.csv', 'data/raw/9894.csv', 'data/raw/9670.csv', 'data/raw/9664.csv', 'data/raw/8618.csv', 'data/raw/9506.csv', 'data/raw/9507.csv', 'data/raw/8619.csv', 'data/raw/9665.csv', 'data/raw/9671.csv', 'data/raw/9659.csv', 'data/raw/9317.csv', 'data/raw/9471.csv', 'data/raw/9129.csv', 'data/raw/9667.csv', 'data/raw/9673.csv', 'data/raw/9505.csv', 'data/raw/9504.csv', 'data/raw/9672.csv', 'data/raw/9666.csv', 'data/raw/9470.csv', 'data/raw/9662.csv', 'data/raw/9676.csv', 'data/raw/9501.csv', 'data/raw/9677.csv', 'data/raw/9688.csv', 'data/raw/9461.csv', 'data/raw/9477.csv', 'data/raw/E048.csv', 'data/raw/9675.csv', 'data/raw/9107.csv', 'data/raw/9661.csv', 'data/raw/9503.csv', 'data/raw/8620.csv', 'data/raw/9502.csv', 'data/raw/9106.csv', 'data/raw/9660.csv', 'data/raw/9674.csv', 'data/raw/9476.csv', 'data/raw/E006.csv', 'data/raw/9411.csv', 'data/raw/9149.csv', 'data/raw/9161.csv', 'data/raw/9217.csv', 'data/raw/9148.csv', 'data/raw/9410.csv', 'data/raw/E007.csv', 'data/raw/94

In [42]:
total_filled = 0

# Do the processing against all the files.
for file in files:
    # Make sure to load the data.
    df = load_data(file)
    if df is None:
        continue

    # Computes metrics.
    total_len, index_len = evaluate_index_size(df, verbose=False)
    total_filled += total_len - index_len

    # Fill the gaps.
    df = fill_gaps(df)

    # Write it back.
    df.to_csv(
        os.path.join(DATA_PATH_OUT, os.path.split(file)[-1]), index_label=COLUMN_DATE
    )

print(f"Filled {total_filled} rows across {len(files)} porfolios!")

Filled 148195 rows across 156 porfolios!
